## Deliverable 2. Create a Customer Travel Destinations Map.

In [20]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vanimo,PG,-2.6741,141.3028,83.82,72,83,2.86,broken clouds
1,1,Tuatapere,NZ,-46.1333,167.6833,49.82,77,91,8.34,overcast clouds
2,2,Olinda,BR,-8.0089,-34.8553,74.93,83,75,8.81,broken clouds
3,3,Hilo,US,19.7297,-155.0900,82.80,69,1,12.66,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,72.63,66,83,13.87,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vanimo,PG,-2.6741,141.3028,83.82,72,83,2.86,broken clouds
3,3,Hilo,US,19.7297,-155.0900,82.80,69,1,12.66,clear sky
6,6,Zumarraga,PH,11.6388,124.8417,81.30,81,100,3.20,overcast clouds
15,15,Butaritari,KI,3.0707,172.7902,83.26,74,94,10.25,overcast clouds
17,17,Isangel,VU,-19.5500,169.2667,77.11,73,40,13.09,scattered clouds
18,18,Samarai,PG,-10.6167,150.6667,79.54,86,85,9.04,overcast clouds
20,20,Garden City,US,42.3256,-83.3310,80.85,59,40,1.01,scattered clouds
21,21,Moussoro,TD,13.6408,16.4901,89.92,31,8,8.50,clear sky
29,29,Kapaa,US,22.0752,-159.3190,82.72,77,90,1.99,moderate rain
37,37,Deoria,IN,26.5047,83.7872,87.37,52,74,4.41,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                225
City                   225
Country                225
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vanimo,PG,83.82,broken clouds,-2.6741,141.3028,
3,Hilo,US,82.80,clear sky,19.7297,-155.0900,
6,Zumarraga,PH,81.30,overcast clouds,11.6388,124.8417,
15,Butaritari,KI,83.26,overcast clouds,3.0707,172.7902,
17,Isangel,VU,77.11,scattered clouds,-19.5500,169.2667,
18,Samarai,PG,79.54,overcast clouds,-10.6167,150.6667,
20,Garden City,US,80.85,scattered clouds,42.3256,-83.3310,
21,Moussoro,TD,89.92,clear sky,13.6408,16.4901,
29,Kapaa,US,82.72,moderate rain,22.0752,-159.3190,
37,Deoria,IN,87.37,broken clouds,26.5047,83.7872,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vanimo,PG,83.82,broken clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
3,Hilo,US,82.80,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Zumarraga,PH,81.30,overcast clouds,11.6388,124.8417,Alegria
15,Butaritari,KI,83.26,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
17,Isangel,VU,77.11,scattered clouds,-19.5500,169.2667,Tanna Lodge
...,...,...,...,...,...,...,...
684,Itoman,JP,86.40,broken clouds,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa
685,Tual,ID,81.88,broken clouds,-5.6667,132.7500,Grand Vilia Hotel
689,Honiara,SB,83.26,overcast clouds,-9.4333,159.9500,Heritage Park Hotel
690,Pekan,MY,78.91,overcast clouds,3.4836,103.3996,Pekan Budget Hotel


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [22]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vanimo,PG,83.82,broken clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
3,Hilo,US,82.80,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Zumarraga,PH,81.30,overcast clouds,11.6388,124.8417,Alegria
15,Butaritari,KI,83.26,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
17,Isangel,VU,77.11,scattered clouds,-19.5500,169.2667,Tanna Lodge
...,...,...,...,...,...,...,...
682,Simoes,BR,79.20,few clouds,-7.5989,-40.8178,Hotel Ponto Certo
684,Itoman,JP,86.40,broken clouds,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa
685,Tual,ID,81.88,broken clouds,-5.6667,132.7500,Grand Vilia Hotel
689,Honiara,SB,83.26,overcast clouds,-9.4333,159.9500,Heritage Park Hotel


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))